# Automação Web e Busca de Informações com Python

#### Desafio: Aula dia 4

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver
- chrome: chromedriver

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

navegador = webdriver.Chrome() # abre o navegador

In [3]:
# pesquisar no google a cotação dólar
# pegar a cotação do dólar
navegador.get('https://www.google.com.br/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# pegando o valor da busca do google
cotacao_dolar = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_dolar)


5.4222


In [4]:
# pesquisar no google a cotação euro
# pegar a cotação do euro

navegador.get('https://www.google.com.br/')
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# pegando o valor da busca do google
cotacao_euro = navegador.find_element_by_xpath('//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

print(cotacao_euro)


6.53799827


In [5]:
# entrar no site melhor câmbio
# pegar a cotação do ouro
site = 'https://www.melhorcambio.com/ouro-hoje'
navegador.get(site)

# pegando o valor da busca do google
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',','.')

print(cotacao_ouro)
    
#navegador.find_element_by_xpath('copy xpath').click()

312.27


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [6]:
import pandas as pd

tabela_produtos = pd.read_excel('Produtos.xlsx')
display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [7]:
# passo 1: Atualizar as cotações
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Dólar','Cotação'] = float(cotacao_dolar)
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Euro','Cotação'] = float(cotacao_euro)
tabela_produtos.loc[tabela_produtos['Moeda'] == 'Ouro','Cotação'] = float(cotacao_ouro)

# passo 2: Atualizar o preço base reais -> cotação + preço base original
tabela_produtos['Preço Base Reais'] = tabela_produtos['Cotação'] * tabela_produtos['Preço Base Original']

# passo 3: Atualizar o preço final -> preço base reais + ajuste
tabela_produtos['Preço Final'] = tabela_produtos['Preço Base Reais'] * tabela_produtos['Ajuste']

# Formatar os valores
tabela_produtos['Preço Base Reais'] = tabela_produtos['Preço Base Reais'].map("{:.2f}".format)
tabela_produtos['Preço Final'] = tabela_produtos['Preço Final'].map("{:.2f}".format) # 2 casas depois do .
tabela_produtos['Cotação'] = tabela_produtos['Cotação'].map("{:.3f}".format) # 3 casas depois do .

display(tabela_produtos)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.422,5422.15,1.40,7591.00
1,Carro Renault,4500.00,Euro,6.538,29420.99,2.00,58841.98
2,Notebook Dell,899.99,Dólar,5.422,4879.93,1.70,8295.87
3,IPhone,799.00,Dólar,5.422,4332.34,1.70,7364.97
4,Carro Fiat,3000.00,Euro,6.538,19613.99,1.90,37266.59
5,Celular Xiaomi,480.48,Dólar,5.422,2605.26,2.00,5210.52
6,Joia 20g,20.00,Ouro,312.270,6245.40,1.15,7182.21


### Agora vamos exportar a nova base de preços atualizada

In [8]:
tabela_produtos.to_excel('Produtos Atualizado.xlsx', index = False) # esse index = False deleta a coluna do index